In [1]:
pip install opendatasets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 KB 117.1 kB/s eta 0:00:00 kB/s eta 0:00:01:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 KB 483.8 kB/s eta 0:00:001m485.9 kB/s eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.6.2-py3-none-any.whl size=111929 sha256=33c704381f17e761328ce29109d631b72be168be5d5ee08e61e551d481eac324
  Stored in directory: /home/bubbles/.cache/pip/wheels/a4/75/8c/257931ed51ad92deed9b0b16a7fde62de102fa3e5e68542cdc
Successfully built kaggle
Note: you may need to restart the kernel to use updated packages.


In [2]:
from torchvision.transforms import transforms
transform=transforms.Compose([transforms.ToTensor(),transforms.Resize(size=(224, 224))])
from torchvision import datasets
BASE_PATH="/home/bubbles/Quantbit Internship/train"
BASE_PATH_TEST="/home/bubbles/Quantbit Internship/test"
train=datasets.ImageFolder(root=BASE_PATH,transform=transform)
test=datasets.ImageFolder(root=BASE_PATH_TEST,transform=transform)

In [4]:
from torch.utils.data import DataLoader
train_loader=DataLoader(train, batch_size=16,shuffle=True)
test_loader=DataLoader(test,batch_size=16,shuffle=False)
from torchvision import models
model=models.vgg16('IMAGENET1K_V1')

In [5]:
for param in model.features.parameters():
    param.requires_grad = False
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
from torch import nn
# model.fc=nn.Linear(1000,2)
# _modules['6']
model.classifier._modules['6']=nn.Linear(4096,2)
# model.classifier._modules['7']=nn.Sigmoid()

In [8]:
from torch.optim import Adam
clf=model.to('cpu')
opt=Adam(clf.parameters(),lr=1e-5)
loss_fn=nn.CrossEntropyLoss()

In [9]:
import torch
for epoch in range(20):
  total_correct = 0
  total_samples = 0
  for batch in train_loader:
    X,y=batch
    X,y=X.to('cpu'),y.to('cpu')
    yhat=clf(X)
    # print(yhat,y.unsqueeze(1))
    # print(yhat.squeeze(1).float(),y)
    loss=loss_fn(yhat,y)

    opt.zero_grad()
    loss.backward()
    opt.step()
    # print(y,yhat)
    _, predicted = torch.max(yhat, 1)

        # Update the running total of correct predictions and samples
    total_correct += (predicted == y).sum().item()
    total_samples += y.size(0)

    # Calculate the accuracy for this epoch
  accuracy = 100 * total_correct / total_samples
  print(f'Epoch {epoch+1}: Accuracy = {accuracy:.2f}% Loss:{loss}')



/home/bubbles/.local/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1: Accuracy = 65.53% Loss:0.46987372636795044
Epoch 2: Accuracy = 91.74% Loss:0.2154947817325592
Epoch 3: Accuracy = 94.61% Loss:0.26392555236816406
Epoch 4: Accuracy = 97.85% Loss:0.15809136629104614
Epoch 5: Accuracy = 98.74% Loss:0.07499127835035324
Epoch 6: Accuracy = 98.38% Loss:0.03909144923090935
Epoch 7: Accuracy = 99.46% Loss:0.039741382002830505
Epoch 8: Accuracy = 100.00% Loss:0.036599092185497284
Epoch 9: Accuracy = 99.82% Loss:0.04811512678861618
Epoch 10: Accuracy = 100.00% Loss:0.013275116682052612
Epoch 11: Accuracy = 100.00% Loss:0.023816168308258057
Epoch 12: Accuracy = 100.00% Loss:0.026023752987384796


KeyboardInterrupt: 

In [11]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images=images.to('cpu')
        labels=labels.to('cpu')
        # calculate outputs by running images through the network
        outputs = clf(images.float())
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

Accuracy of the network on the test images: 95 %
